In [4]:
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

/kaggle/input/jeju-geo/jeju_geo_data.json
/kaggle/input/coronavirusdataset/TimeAge.csv
/kaggle/input/coronavirusdataset/Case.csv
/kaggle/input/coronavirusdataset/Policy.csv
/kaggle/input/coronavirusdataset/Region.csv
/kaggle/input/coronavirusdataset/PatientInfo.csv
/kaggle/input/coronavirusdataset/SearchTrend.csv
/kaggle/input/coronavirusdataset/Weather.csv
/kaggle/input/coronavirusdataset/TimeProvince.csv
/kaggle/input/coronavirusdataset/TimeGender.csv
/kaggle/input/coronavirusdataset/SeoulFloating.csv
/kaggle/input/coronavirusdataset/Time.csv
/kaggle/input/korea-location/korea_choropleth.json


In [5]:
timeAge = pd.read_csv('/kaggle/input/coronavirusdataset/TimeAge.csv')
case = pd.read_csv('/kaggle/input/coronavirusdataset/Case.csv')
policy = pd.read_csv('/kaggle/input/coronavirusdataset/Policy.csv')
region = pd.read_csv('/kaggle/input/coronavirusdataset/Region.csv')
patientInfo = pd.read_csv('/kaggle/input/coronavirusdataset/PatientInfo.csv')
searchTrend = pd.read_csv('/kaggle/input/coronavirusdataset/SearchTrend.csv')
weather = pd.read_csv('/kaggle/input/coronavirusdataset/Weather.csv')
timeProvince = pd.read_csv('/kaggle/input/coronavirusdataset/TimeProvince.csv')
timeGender = pd.read_csv('/kaggle/input/coronavirusdataset/TimeGender.csv')
seoulFloating = pd.read_csv('/kaggle/input/coronavirusdataset/SeoulFloating.csv')
time = pd.read_csv('/kaggle/input/coronavirusdataset/Time.csv')

In [41]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
from matplotlib import rc

# Interactive plotting
import plotly
import plotly.graph_objs as go
#import chart_studio.plotly as py
from plotly import tools
from plotly.offline import download_plotlyjs, init_notebook_mode, plot,iplot


import matplotlib.animation as animation
%matplotlib inline

from IPython.display import set_matplotlib_formats
set_matplotlib_formats("retina")
pd.options.plotting.backend = 'plotly'

import plotly.io as pio
pio.renderers.default = "notebook_connected"

# Covid case over time
- Dataset: Time.csv
- Confirmed case : bar
- Positivity (confirmed case / test): line
- Fatality (deceased / confirmed case): line
- Optional: Bring in Policy.csv to mark specific timeline

In [8]:
time.info()
time = time.drop([0, 1, 2])
# For calculating Covid positive rate
positive_rate = (time.confirmed / time.test ) * 100

# Covid fatality rate
fatality_rate = (time.deceased / time.confirmed) * 100

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 163 entries, 0 to 162
Data columns (total 7 columns):
 #   Column     Non-Null Count  Dtype 
---  ------     --------------  ----- 
 0   date       163 non-null    object
 1   time       163 non-null    int64 
 2   test       163 non-null    int64 
 3   negative   163 non-null    int64 
 4   confirmed  163 non-null    int64 
 5   released   163 non-null    int64 
 6   deceased   163 non-null    int64 
dtypes: int64(6), object(1)
memory usage: 9.0+ KB


In [11]:
data=[]
total_case=[]
positivity=[]
released = []
fatality=[]
test_case = []
date="월-주차"
cols='rgb(0,0,255)'

total_case = go.Bar(
    x = time['date'],
    y = time['confirmed'],
    name="Total confirmed Covid-19 cases",
    opacity = 0.6,
    marker={'color':'LightPink', 'line':{'color':'White','width':1}},
    yaxis='y2'
    )

released_case = go.Bar(
    x = time['date'],
    y = time['released'],
    name="Total released Covid-19 cases",
    opacity = 0.6,
    marker={'color':'SlateBlue', 'line':{'color':'White','width':1}},
    yaxis='y2'
    )

positivity = go.Scatter(
    x = time['date'],
    y = positive_rate,
    name="Covid-19 Positive Rate",
    marker={'color':'#84B1ED', 'line':{'color':'Lavender', 'width':2}}
    )

fatality = go.Scatter(
    x= time['date'],
    y= fatality_rate,
    name="Covid-19 Fatality Rate",
    marker={'color':'Coral', 'line':{'color':'Lavender', 'width':2}}
    )

layout = go.Layout(
    title = '<b>South Korea Daily Covid-19 Cases (Jan.2020 - Jun.2020) </b>',
    hovermode='x unified',
    xaxis = {'title':'Days'},
    yaxis = {'title':'Rate(%)'},
    yaxis2 = {'title':'Covid-19 cases', 'overlaying':'y', 'side':'right'},
    showlegend = True
)

data = [total_case, released_case, positivity, fatality]
fig = go.Figure(data = data, layout=layout)

fig.update_layout(legend=dict(
    yanchor="top",
    y=0.99,
    xanchor="left",
    x=0.04 ),
                  title_x = 0.5
                  
)

# yanchor="top",
#     y=0.99,
#     xanchor="left",
#     x=0.01
    
# orientation="h",
#     yanchor="bottom",
#     y=1.02,
#     xanchor="right",
#     x=1

plot(fig, show_link= False, filename = 'covid_dashboard.html')
fig.show()

# Daily confirmed cases and government policy

In [12]:
# Calculate daily confirmed cases by subtracting yesterday's confirmed cases from today's confirmed cases 
daily_confirmed = [0]

for i in range(4, len(time.confirmed) + 3):
    daily_confirmed.append(time['confirmed'][i] - time['confirmed'][i - 1])
    
len(daily_confirmed)

160

In [13]:
import plotly.express as px

# Plot daily confirmed cases

data=[]
daily_case=[]

date="월-주차"
cols='rgb(0,0,255)'

daily_case = go.Bar(
    x = time['date'],
    y = daily_confirmed,
    name="Daily confirmed Covid-19 cases",
    opacity = 0.6,
    marker={'color':'Coral', 'line':{'color':'White','width':1}},
    yaxis='y2'
    )

# markers = px.scatter( 
#     x = policy['start_date'], 
#     y = daily_confirmed, 
#     text = policy["gov_policy"], 
#     log_x=True, 
#     size_max=60
# )


layout = go.Layout(
    title = '<b>Daily Covid-19 cases of South Korea (Jan.2020 - Jun.2020) </b>',
    hovermode='x unified',
    xaxis = {'title':'Days'},
    yaxis = {'title':'Rate(%)'},
    showlegend = False)

data = [daily_case]
fig = go.Figure(data = data, layout=layout)
# plot(fig, show_link= False, filename = 'covid_dashboard.html')
fig.show()

In [14]:
# How to incorporate government policy into daily confirmed cases..?
time['daily_confirmed'] = 0
time['daily_confirmed'] = daily_confirmed

important_dates = ['2020-02-23','2020-02-26', '2020-02-29', '2020-03-02', '2020-03-08','2020-05-27'] 
policy[policy['start_date'] == '2020-03-02'].gov_policy.iloc[0]
policy[policy['start_date'] == '2020-03-08']
getTime = time[time['date'] == important_dates[0]]
getTime['daily_confirmed']

policy[policy['start_date'] == important_dates[0]].gov_policy.item()

'Infectious Disease Alert Level'

In [15]:
import plotly.express as px

# Plot daily confirmed cases

data=[]
daily_case=[]

date="월-주차"
cols='rgb(0,0,255)'

daily_case = go.Bar(
    x = time['date'],
    y = daily_confirmed,
    name="Daily confirmed Covid-19 cases",
    opacity = 0.6,
    marker={'color':'Coral', 'line':{'color':'White','width':1}},
    yaxis='y2'
    )

layout = go.Layout(
    title = '<b>Daily Covid-19 cases of South Korea (Jan.2020 - Jun.2020) </b>',
    hovermode='x unified',
    xaxis = {'title':'Days'},
    yaxis = {'title':'Rate(%)'},
    #yaxis2 = {'title':'Covid-19 cases', 'overlaying':'y', 'side':'right'},
    showlegend = True)

data = [daily_case]
fig = go.Figure(data = data, layout=layout)

for pol in range(len(important_dates)):
    fig.add_annotation(
        x = important_dates[pol], 
        y = time[time['date'] == important_dates[pol]].daily_confirmed.item(),
        text = policy[policy['start_date'] == important_dates[pol]].gov_policy.iloc[0],
        showarrow=True,
        arrowhead=1
    )
# fig.add_annotation(
#         x = '2020-05-28', 
#         y = 300,
#         text = 'sadfd',
#         showarrow=True,
#         arrowhead=1
#     )
fig.update_layout(showlegend=True)
# plot(fig, show_link= False, filename = 'covid_dashboard.html')
fig.show()

## Seoul's Covid Confirmed / Released cases

In [16]:
timeProvince

,date,time,province,confirmed,released,deceased
0,2020-01-20,16,Seoul,0,0,0
1,2020-01-20,16,Busan,0,0,0
2,2020-01-20,16,Daegu,0,0,0
3,2020-01-20,16,Incheon,1,0,0
4,2020-01-20,16,Gwangju,0,0,0
...,...,...,...,...,...,...
2766,2020-06-30,0,Jeollabuk-do,27,21,0
2767,2020-06-30,0,Jeollanam-do,24,19,0
2768,2020-06-30,0,Gyeongsangbuk-do,1389,1328,54
2769,2020-06-30,0,Gyeongsangnam-do,134,128,0


In [17]:
seoul_confirmed = timeProvince[timeProvince['province'] == 'Seoul']
seoul_confirmed = seoul_confirmed.reset_index()
seoul_confirmed['confirmed'][5]

0

In [18]:
# Calculate daily confirmed cases by subtracting yesterday's confirmed cases from today's confirmed cases 
seoul_daily_confirmed = [0]
seoul_confirmed_dates = ['2020-01-20']

for i in range(7, len(seoul_confirmed.confirmed) - 1, 7):
    seoul_confirmed_dates.append(seoul_confirmed.date[i])
    seoul_daily_confirmed.append(seoul_confirmed['confirmed'][i] - seoul_confirmed['confirmed'][i - 7])
    
# Calculate daily released cases by subtracting yesterday's released cases from today's released cases 
seoul_daily_released = [0]

for i in range(7, len(seoul_confirmed.released) -1, 7):
    seoul_daily_released.append(seoul_confirmed['released'][i] - seoul_confirmed['released'][i - 7])

print(len(seoul_daily_confirmed))
print(len(seoul_daily_released))

24
24


In [19]:
import plotly.express as px

# Plot daily confirmed cases

data=[]
daily_case=[]
released_case = []

cols='rgb(0,0,255)'

daily_case = go.Bar(
    x = seoul_confirmed_dates,
    y = seoul_daily_confirmed,
    name="Confirmed",
    opacity = 0.7,
    marker={'color':'LightCoral', 'line':{'color':'White','width':1.4}},
    yaxis='y2',
    #mode='lines+markers'
)

released_case = go.Bar(
    x = seoul_confirmed_dates,
    y = seoul_daily_released,
    name="Released",
    opacity = 0.7,
    marker={'color':'SlateBlue', 'line':{'color':'White','width':1.4}},
    yaxis='y2',
    #mode='lines+markers'
)

layout = go.Layout(
    title = '<b>Weekly Covid-19 cases of Seoul (Jan.2020 - Jun.2020) </b>',
    hovermode='x unified',
    xaxis = {'title':'Days'},
    yaxis = {'title':'Rate(%)'},
    #yaxis2 = {'title':'Covid-19 cases', 'overlaying':'y', 'side':'right'},
    showlegend = True)

data = [daily_case, released_case]
fig = go.Figure(data = data, layout=layout)

fig.update_layout(showlegend=True)
fig.update_layout(legend=dict(
    orientation="h",
    yanchor="bottom",
    y=1.02,
    xanchor="right",
    x=1),
                  title_x = 0.5
                  
)
# plot(fig, show_link= False, filename = 'covid_dashboard.html')
fig.show()

In [20]:
import plotly.express as px

data=[]
daily_case=[]
released_case = []

daily_case = go.Bar(
    x = seoul_confirmed_dates,
    y = seoul_daily_confirmed,
    name="Confirmed",
    opacity = 0.5,
    marker={'color':'LightCoral', 'line':{'color':'White','width':1.4}},
    yaxis='y2',
)

released_case = go.Bar(
    x = seoul_confirmed_dates,
    y = seoul_daily_released,
    name="Released",
    opacity = 0.5,
    marker={'color':'SlateBlue', 'line':{'color':'White','width':1.4}},
    yaxis='y2',
)

layout = go.Layout(
    title = '<b>Weekly Covid-19 Confirmed & Released cases of Seoul (Jan.2020 - Jun.2020) </b>',
    hovermode='x unified',
    xaxis = {'title':'Weeks'},
    yaxis = {'title' : 'Confirmed Cases'},
    yaxis2 = {'title':'Released Cases'},
    showlegend = True, 
    updatemenus=[
        dict(
            type='buttons', showactive=False,
            y=1.05,
            x=1.15,
            xanchor='right',
            yanchor='middle',
            pad=dict(t=0, r=10),
            buttons=[dict(label='Play',
            method='animate',
            args=[None, 
                  dict(frame=dict(duration=40, 
                                  redraw=False),
                                  transition=dict(duration=40000),
                                  fromcurrent=True,
                                  mode='immediate')]
                     )])]

)

frames = [dict(data= [dict(type='bar',
                           x = seoul_confirmed_dates[:k + 1],
                           y = seoul_daily_confirmed[:k + 1]),
                      dict(type='bar',
                           x = seoul_confirmed_dates[:k + 1],
                           y = seoul_daily_released[:k + 1])
                     ],
               #traces= [0,1,2,3],  
              )for k in range(1, len(seoul_confirmed_dates))] 

data = [daily_case, released_case]
fig = go.Figure(data = data, layout=layout)

fig = tools.make_subplots(rows=1, cols=2)
fig.append_trace(daily_case, 1, 1)
fig.append_trace(released_case, 1, 2)

fig.update_layout(legend=dict(
    orientation="h",
    yanchor="bottom",
    y=1.02,
    xanchor="right",
    x=1),
                  title_x = 0.5,
                          
)

fig.update(frames = frames,layout=layout)

# plot(fig, show_link= False, filename = 'covid_dashboard.html')
fig.show()

/opt/conda/lib/python3.7/site-packages/plotly/tools.py:465: DeprecationWarning:

plotly.tools.make_subplots is deprecated, please use plotly.subplots.make_subplots instead



### Confirmed Cases by South Korea's province

In [21]:
timeProvinceNew = timeProvince.copy()

for i in range(len(timeProvinceNew)):
    timeProvinceNew.date[i] = timeProvinceNew.date[i][0:7]

/opt/conda/lib/python3.7/site-packages/ipykernel_launcher.py:4: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy



In [22]:
timeProvince

,date,time,province,confirmed,released,deceased
0,2020-01-20,16,Seoul,0,0,0
1,2020-01-20,16,Busan,0,0,0
2,2020-01-20,16,Daegu,0,0,0
3,2020-01-20,16,Incheon,1,0,0
4,2020-01-20,16,Gwangju,0,0,0
...,...,...,...,...,...,...
2766,2020-06-30,0,Jeollabuk-do,27,21,0
2767,2020-06-30,0,Jeollanam-do,24,19,0
2768,2020-06-30,0,Gyeongsangbuk-do,1389,1328,54
2769,2020-06-30,0,Gyeongsangnam-do,134,128,0


In [23]:
timeProvinceNew.province.unique()

array(['Seoul', 'Busan', 'Daegu', 'Incheon', 'Gwangju', 'Daejeon',
       'Ulsan', 'Sejong', 'Gyeonggi-do', 'Gangwon-do',
       'Chungcheongbuk-do', 'Chungcheongnam-do', 'Jeollabuk-do',
       'Jeollanam-do', 'Gyeongsangbuk-do', 'Gyeongsangnam-do', 'Jeju-do'],
      dtype=object)

In [24]:
# Merge daily confirmed/released cases by months

province = ['Seoul', 'Busan', 'Daegu', 'Incheon', 'Gwangju', 'Daejeon',
       'Ulsan', 'Sejong', 'Gyeonggi-do', 'Gangwon-do',
       'Chungcheongbuk-do', 'Chungcheongnam-do', 'Jeollabuk-do',
       'Jeollanam-do', 'Gyeongsangbuk-do', 'Gyeongsangnam-do', 'Jeju-do']

# For storing every months' confirmed cases of each province
months_df = ['2020-01' for _ in range(len(province))] + ['2020-02' for _ in range(len(province))] + ['2020-03' for _ in range(len(province))] + ['2020-04' for _ in range(len(province))] + ['2020-05' for _ in range(len(province))]+  ['2020-06' for _ in range(len(province))]
months = ['2020-01', '2020-02', '2020-03', '2020-04', '2020-05', '2020-06']
confirmed = []

for m in months:
    for p in province:
        lendf = len(timeProvinceNew[(timeProvinceNew['date'] == '{}'.format(m)) & (timeProvinceNew['province'] == '{}'.format(p))])
        firstcase = timeProvinceNew[(timeProvinceNew['date'] == '{}'.format(m)) & (timeProvinceNew['province'] == '{}'.format(p))].reset_index()['confirmed'][0]
        lastcase = timeProvinceNew[(timeProvinceNew['date'] == '{}'.format(m)) & (timeProvinceNew['province'] == '{}'.format(p))].reset_index()['confirmed'][lendf - 1]
        monthcase = lastcase - firstcase
        confirmed.append(monthcase)

In [25]:
# Create a dataframe
data = {'date':  months_df,
        'province': province * 6,
        'confirmed': confirmed
        }
months_provinces = pd.DataFrame(data = data, columns = ['date', 'province', 'confirmed'])
months_provinces

,date,province,confirmed
0,2020-01,Seoul,4
1,2020-01,Busan,0
2,2020-01,Daegu,0
3,2020-01,Incheon,0
4,2020-01,Gwangju,0
...,...,...,...
97,2020-06,Jeollabuk-do,6
98,2020-06,Jeollanam-do,4
99,2020-06,Gyeongsangbuk-do,10
100,2020-06,Gyeongsangnam-do,11


In [26]:
def draw_barchart(current_month):
    # Get current month's dataframe
    curr_month_df = months_provinces[months_provinces['date'] == '2020-0{}'.format(current_month)]
    
    # Draw graph
    data=[]
    confirmed=[]
    
    confirmed = go.Bar(
    x = curr_month_df['province'],
    y = curr_month_df['confirmed'],
    name="Covid-19 Confirmed cases by Province",
    opacity = 0.6,
    marker={'color':'LightPink', 'line':{'color':'White','width':1}},
    yaxis='y'
    )
    
    layout = go.Layout(
    title = '<b>South Korea Covid-19 Confirmed cases by Province ({}월) </b>'.format(current_month),
    hovermode='x unified',
    xaxis = {'title':'Province'},
    yaxis = {'title':'Cases'},
    #yaxis2 = {'title':'Covid-19 cases', 'overlaying':'y', 'side':'right'},
    showlegend = True)

    data = [confirmed]
    fig = go.Figure(data = data, layout=layout)
    
    fig.update_layout(showlegend=False)
    # plot(fig, show_link= False, filename = 'covid_dashboard.html')
    fig.show()

In [28]:
# For user input
draw_barchart(int(input('Enter month:')))

Enter month: 3


### Folium Documentation: 
- [Jupyter Documentation](https://nbviewer.jupyter.org/github/python-visualization/folium/blob/master/examples/Plugins.ipynb#Timestamped-GeoJSON)
- [Kaggle Example](https://www.kaggle.com/daveianhickey/how-to-folium-for-maps-heatmaps-time-analysis)

In [29]:
months_provinces

,date,province,confirmed
0,2020-01,Seoul,4
1,2020-01,Busan,0
2,2020-01,Daegu,0
3,2020-01,Incheon,0
4,2020-01,Gwangju,0
...,...,...,...
97,2020-06,Jeollabuk-do,6
98,2020-06,Jeollanam-do,4
99,2020-06,Gyeongsangbuk-do,10
100,2020-06,Gyeongsangnam-do,11


In [30]:
# Libraries for folium
import json
import folium

mapp = folium.Map(location=[37.566345, 126.977893])   # Center as Seoul City Hall
mapp

In [32]:
# Load json file
geo_json = json.load(open('/kaggle/input/korea-location/korea_choropleth.json', encoding="utf-8"))

In [33]:
months_provinces

,date,province,confirmed
0,2020-01,Seoul,4
1,2020-01,Busan,0
2,2020-01,Daegu,0
3,2020-01,Incheon,0
4,2020-01,Gwangju,0
...,...,...,...
97,2020-06,Jeollabuk-do,6
98,2020-06,Jeollanam-do,4
99,2020-06,Gyeongsangbuk-do,10
100,2020-06,Gyeongsangnam-do,11


In [34]:
months_provinces.province.unique()

array(['Seoul', 'Busan', 'Daegu', 'Incheon', 'Gwangju', 'Daejeon',
       'Ulsan', 'Sejong', 'Gyeonggi-do', 'Gangwon-do',
       'Chungcheongbuk-do', 'Chungcheongnam-do', 'Jeollabuk-do',
       'Jeollanam-do', 'Gyeongsangbuk-do', 'Gyeongsangnam-do', 'Jeju-do'],
      dtype=object)

In [35]:
months_provinces_grouped = months_provinces.groupby(['province']).sum()
months_provinces_grouped

,confirmed
province,
Busan,148
Chungcheongbuk-do,64
Chungcheongnam-do,151
Daegu,6417
Daejeon,117
Gangwon-do,55
Gwangju,40
Gyeonggi-do,1162
Gyeongsangbuk-do,1318


In [36]:
center = [36.8000, 127.7000] # Center of South Korea
title_html = '''
             <h3 align="center" style="font-size:40px"><b>Your map title</b></h3>
             '''
map = folium.Map(location=center, zoom_start=7, titles = title_html)

map.choropleth(geo_data = geo_json,
              data = months_provinces_grouped['confirmed'],
              columns = [months_provinces_grouped.index, months_provinces_grouped['confirmed']],
              fill_color="YlGnBu",
              key_on="feature.properties.CTP_ENG_NM"
              )

map

/opt/conda/lib/python3.7/site-packages/folium/folium.py:413: FutureWarning:

The choropleth  method has been deprecated. Instead use the new Choropleth class, which has the same arguments. See the example notebook 'GeoJSON_and_choropleth' for how to do this.



In [37]:
region

,code,province,city,latitude,longitude,elementary_school_count,kindergarten_count,university_count,academy_ratio,elderly_population_ratio,elderly_alone_ratio,nursing_home_count
0,10000,Seoul,Seoul,37.566953,126.977977,607,830,48,1.44,15.38,5.8,22739
1,10010,Seoul,Gangnam-gu,37.518421,127.047222,33,38,0,4.18,13.17,4.3,3088
2,10020,Seoul,Gangdong-gu,37.530492,127.123837,27,32,0,1.54,14.55,5.4,1023
3,10030,Seoul,Gangbuk-gu,37.639938,127.025508,14,21,0,0.67,19.49,8.5,628
4,10040,Seoul,Gangseo-gu,37.551166,126.849506,36,56,1,1.17,14.39,5.7,1080
...,...,...,...,...,...,...,...,...,...,...,...,...
239,61160,Gyeongsangnam-do,Haman-gun,35.272481,128.406540,16,20,0,1.19,23.74,14.7,94
240,61170,Gyeongsangnam-do,Hamyang-gun,35.520541,127.725177,13,12,0,1.01,32.65,20.9,83
241,61180,Gyeongsangnam-do,Hapcheon-gun,35.566702,128.165870,17,15,0,0.71,38.44,24.7,96
242,70000,Jeju-do,Jeju-do,33.488936,126.500423,113,123,4,1.53,15.10,6.4,1245


## Confirmed cases and their location
- Choropleth map
- Marker clusters

In [38]:
# Delete string values from lat, lon and convert datatypes to float
case_nonull = case[case.latitude != '-']
case_nonull.latitude = case_nonull.latitude.astype(float)
case_nonull.longitude = case_nonull.longitude.astype(float)
case_nonull.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 65 entries, 0 to 164
Data columns (total 8 columns):
 #   Column          Non-Null Count  Dtype  
---  ------          --------------  -----  
 0    case_id        65 non-null     int64  
 1   province        65 non-null     object 
 2   city            65 non-null     object 
 3   group           65 non-null     bool   
 4   infection_case  65 non-null     object 
 5   confirmed       65 non-null     int64  
 6   latitude        65 non-null     float64
 7   longitude       65 non-null     float64
dtypes: bool(1), float64(2), int64(2), object(3)
memory usage: 4.1+ KB


/opt/conda/lib/python3.7/site-packages/pandas/core/generic.py:5494: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy



In [39]:
from folium.plugins import MarkerCluster

center = [36.8000, 127.7000] # Center of South Korea

mapp = folium.Map(location=center, zoom_start = 7)

# Marker cluster
marker_cluster = MarkerCluster(name='Confirmed locations - Detail',).add_to(mapp)
for row in case_nonull.itertuples():
    folium.Marker(location=[row.latitude,row.longitude],popup=row.infection_case).add_to(marker_cluster)
    
# Choropleth map
folium.Choropleth(geo_data = geo_json,
              data = months_provinces_grouped['confirmed'],
              columns = [months_provinces_grouped.index, months_provinces_grouped['confirmed']],
              fill_color="YlGnBu",
              key_on="feature.properties.CTP_ENG_NM",
                  name='Confirmed cases',
              ).add_to(mapp)

folium.LayerControl(collapsed=False).add_to(mapp)
mapp